# Yves KOUAKOU 



# **Microsoft Azure Machine Learning et Power BI : Modèle prédictif **

Nous utilisons le jeu de données Diabète disponible dans Azure Open Datasets.

##### **Importer les données**
##### 

In [21]:
import numpy as np

from azureml.opendatasets import Diabetes

diabetes = Diabetes.get_tabular_dataset()
X = diabetes.drop_columns("Y")
y = diabetes.keep_columns("Y")
X_df = X.to_pandas_dataframe()
y_df = y.to_pandas_dataframe()
X_df

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
0,59,2,32.1,101.00,157,93.2,38.0,4.00,4.8598,87
1,48,1,21.6,87.00,183,103.2,70.0,3.00,3.8918,69
2,72,2,30.5,93.00,156,93.6,41.0,4.00,4.6728,85
3,24,1,25.3,84.00,198,131.4,40.0,5.00,4.8903,89
4,50,1,23.0,101.00,192,125.4,52.0,4.00,4.2905,80
...,...,...,...,...,...,...,...,...,...,...
437,60,2,28.2,112.00,185,113.8,42.0,4.00,4.9836,93
438,47,2,24.9,75.00,225,166.0,42.0,5.00,4.4427,102
439,60,2,24.9,99.67,162,106.6,43.0,3.77,4.1271,95
440,36,1,30.0,95.00,201,125.2,42.0,4.79,5.1299,85


**Construire et entraîner le modèle**

Nous entrainons un modèle de régression ridge 

In [3]:
import joblib
from sklearn.linear_model import Ridge

model = Ridge().fit(X_df,y_df)
joblib.dump(model, 'sklearn_regression_model.pkl')

['sklearn_regression_model.pkl']

###### Enregistrer le modèle

En plus du contenu du fichier de modèle lui-même, votre modèle enregistré stockera des métadonnées. 
Les métadonnées sont utiles lorsque vous gérez et déployez des modèles dans votre espace de travail. 


In [19]:
import sklearn

from azureml.core import Workspace
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

ws = Workspace.from_config()

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Nom du modèle enregistré dans votre espace de travail
                       model_path='./sklearn_regression_model.pkl',  # Fichier local à télécharger et à enregistrer comme modèle.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework utilisé pour créer le modèle.
                       model_framework_version=sklearn.__version__,  # Version de scikit-learn utilisée pour créer le modèle
                       sample_input_dataset=X,
                       sample_output_dataset=y,
                       resource_configuration=ResourceConfiguration(cpu=2, memory_in_gb=4),
                       description='Modèle de régression ridge pour prédire la progression du diabète.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model my-sklearn-model
Name: my-sklearn-model
Version: 2


###### Définir le script de scoring
Lorsque vous déployez un modèle qui sera intégré dans **Power BI**, vous devez définir un script de scoring Python et un environnement personnalisé. Le script de scoring contient deux fonctions :

- La fonction **init()** s'exécute au démarrage du service. Elle charge le modèle (qui est automatiquement téléchargé depuis le registre des modèles) et le désérialise
- La fonction **run(data)** s'exécute lorsqu'un appel au service inclut des données d'entrée qui doivent être évaluées.
  

In [5]:
%%writefile score.py

import json
import pickle
import numpy as np
import pandas as pd
import os
import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


def init():
    global model
    # Remplacer le nom du fichier si nécessaire.
    path = os.getenv('AZUREML_MODEL_DIR') 
    model_path = os.path.join(path, 'sklearn_regression_model.pkl')
    # Désérialise le fichier modèle en un modèle sklearn.
    model = joblib.load(model_path)


input_sample = pd.DataFrame(data=[{
    "AGE": 5,
    "SEX": 2,
    "BMI": 3.1,
    "BP": 3.1,
    "S1": 3.1,
    "S2": 3.1,
    "S3": 3.1,
    "S4": 3.1,
    "S5": 3.1,
    "S6": 3.1
}])

# Il s'agit d'un échantillon de type entier. Vous pouvez utiliser le type de données qui reflète le résultat attendu.
output_sample = np.array([0])

# Pour indiquer que nous supportons une longueur variable d'entrée de données,
# définir enforce_shape=False
@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        print("input_data....")
        print(data.columns)
        print(type(data))
        result = model.predict(data)
        print("result.....")
        print(result)
    # Vous pouvez renvoyer n'importe quel type de données, tant qu'elles peuvent être sérialisées par JSON.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Writing score.py


###### Définir l'environnement personnalisé
Ensuite, définissez l'environnement de notation du modèle. 
Dans l'environnement, définissez les packages Python, tels que pandas et scikit-learn, dont le script de notation (**score.py**) a besoin.

In [6]:
from azureml.core.model import InferenceConfig
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'pandas',
    'scikit-learn=={}'.format(sklearn.__version__)
])

inference_config = InferenceConfig(entry_script='./score.py',environment=environment)

###### Déployer le modèle


In [7]:
service_name = 'my-diabetes-model'

service = Model.deploy(ws, service_name, [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-30 22:42:20+00:00 Creating Container Registry if not exists..
2022-09-30 22:52:21+00:00 Registering the environment..
2022-09-30 22:52:22+00:00 Building image..
2022-09-30 23:02:37+00:00 Generating deployment configuration.
2022-09-30 23:02:38+00:00 Submitting deployment to compute..
2022-09-30 23:02:43+00:00 Checking the status of deployment my-diabetes-model..
2022-09-30 23:04:43+00:00 Checking the status of inference endpoint my-diabetes-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [8]:
import json

input_payload = json.dumps({
    'data': X_df[0:2].values.tolist()
})

output = service.run(input_payload)

print(output)

[[205.59094435613133], [68.84146418576978]]
